In [95]:
!wget -O dataset.zip "https://www.dropbox.com/scl/fo/y1oi3efi157ljjkz7m3sj/API62T6BzDOQyI96NLaIRQQ?dl=0&e=1&preview=database.sqlite&rlkey=okvow1oa13alowespgpgqtlmk"

--2024-12-06 21:16:57--  https://www.dropbox.com/scl/fo/y1oi3efi157ljjkz7m3sj/API62T6BzDOQyI96NLaIRQQ?dl=0&e=1&preview=database.sqlite&rlkey=okvow1oa13alowespgpgqtlmk
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc99da94c11ffb51535cfd6d9bfc.dl.dropboxusercontent.com/zip_download_get/CDLVxtHtB8mYfkzKhPccpXjsyaTdeLAmmT-PCdaaMW01Izd2HL3yuja74Jyhpa38__TA8WgzI0nnmWPWUnAmuzAQv_Ets4C4-YuVe5ePH7oBQw# [following]
--2024-12-06 21:16:58--  https://uc99da94c11ffb51535cfd6d9bfc.dl.dropboxusercontent.com/zip_download_get/CDLVxtHtB8mYfkzKhPccpXjsyaTdeLAmmT-PCdaaMW01Izd2HL3yuja74Jyhpa38__TA8WgzI0nnmWPWUnAmuzAQv_Ets4C4-YuVe5ePH7oBQw
Resolving uc99da94c11ffb51535cfd6d9bfc.dl.dropboxusercontent.com (uc99da94c11ffb51535cfd6d9bfc.dl.dropboxusercontent.com)... 162.125.1.15, 2620:100:6016:15::a27d:10f
Connecting to uc99d

In [96]:
import zipfile

with zipfile.ZipFile("dataset.zip", "r") as zip_ref:
    zip_ref.extractall("./dataset")


In [97]:
!pip install pandas
!pip install sklearn
!pip install scikit-learn
!pip install xgboost

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
    

In [98]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor 
from sklearn.linear_model import LinearRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error 
from math import sqrt
from datetime import datetime

In [99]:
#read data from file
cnx = sqlite3.connect('/kaggle/working/dataset/database.sqlite')
player_data = pd.read_sql_query("SELECT * FROM Player_Attributes", cnx)
player_data.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [100]:
#devide data into numerical and categorical
categorical_attributes = ['preferred_foot', 'attacking_work_rate', 'defensive_work_rate']

numercial_attributes = ['potential', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes']

target_attribute = ['overall_rating']

In [101]:
#size of database
player_data.shape


(183978, 42)

In [102]:
#type of columns
player_data.dtypes

id                       int64
player_fifa_api_id       int64
player_api_id            int64
date                    object
overall_rating         float64
potential              float64
preferred_foot          object
attacking_work_rate     object
defensive_work_rate     object
crossing               float64
finishing              float64
heading_accuracy       float64
short_passing          float64
volleys                float64
dribbling              float64
curve                  float64
free_kick_accuracy     float64
long_passing           float64
ball_control           float64
acceleration           float64
sprint_speed           float64
agility                float64
reactions              float64
balance                float64
shot_power             float64
jumping                float64
stamina                float64
strength               float64
long_shots             float64
aggression             float64
interceptions          float64
positioning            float64
vision  

In [103]:
"""
preprocessing of attributes 

"""


from sklearn.base import TransformerMixin, BaseEstimator


class Clean_and_Merge(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, df, y=None):
        return self
    
    def transform(self, df, y=None):
        categorical_attributes = ['preferred_foot', 'attacking_work_rate', 'defensive_work_rate']
        numercial_attributes = ['potential', 'crossing', 'finishing', 'heading_accuracy',
            'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
            'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
            'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
            'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
            'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
            'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
            'gk_reflexes']
        player_id = df['player_fifa_api_id'].unique()
    
        new_df = pd.DataFrame()

        for i in player_id:
            index = np.where(df['player_fifa_api_id'] == i)[0]

            if len(index) == 0:  # Skip if no rows found
                print(f"No data found for player_id {i}")
                continue

            temp_num = df.iloc[index][numercial_attributes].mean()

            temp_mode = df.iloc[index][categorical_attributes].mode()
            if not temp_mode.empty:
                temp_cat = temp_mode.iloc[0]
            else:
                temp_cat = pd.Series([None] * len(categorical_attributes), index=categorical_attributes)

            temp_df = pd.DataFrame(data=[temp_num.values], columns=temp_num.index)
            temp_df[temp_cat.index] = temp_cat.values

            new_df = pd.concat([new_df, temp_df], ignore_index=True)

        return new_df.reset_index(drop=True)
    
class Clean_and_Merge_Target(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, df, y=None):
        return self
    
    def transform(self, df, y=None):
        df.dropna(thresh=10, inplace=True)
        player_id = df['player_fifa_api_id'].unique()
        
        new_df = pd.DataFrame()

        for i in player_id:
            index = np.where(df['player_fifa_api_id'] == i)[0]

            temp_num = df.iloc[index][target_attribute].mean()

            temp_df = pd.DataFrame(data=[temp_num.values], columns=temp_num.index)

            new_df = pd.concat([new_df, temp_df], ignore_index=True)
            
        
        return new_df.reset_index(drop=True)
    


 

In [104]:
# preprocessing for target : rate of every player in every activity in match


custom_attribute_object = Clean_and_Merge()
merged_player_data = custom_attribute_object.fit_transform(player_data)
print(merged_player_data.head())

   potential   crossing  finishing  heading_accuracy  short_passing  \
0  67.600000  48.600000  43.600000         70.600000      60.600000   
1  74.484848  70.787879  49.454545         52.939394      62.272727   
2  74.192308  68.115385  57.923077         58.692308      65.115385   
3  70.782609  57.217391  26.260870         69.260870      64.695652   
4  74.680000  45.080000  38.840000         73.040000      64.760000   

     volleys  dribbling      curve  free_kick_accuracy  long_passing  ...  \
0  43.600000  50.600000  44.600000           38.600000     63.600000  ...   
1  29.151515  61.090909  61.878788           62.121212     63.242424  ...   
2  54.269231  69.038462  60.192308           55.615385     60.461538  ...   
3  47.782609  55.565217  37.782609           40.391304     60.826087  ...   
4  32.080000  50.600000  45.480000           26.360000     56.840000  ...   

   standing_tackle  sliding_tackle  gk_diving  gk_handling  gk_kicking  \
0        66.000000       67.800000  

In [105]:
merged_player_data

,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,...,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,preferred_foot,attacking_work_rate,defensive_work_rate
0,67.600000,48.600000,43.600000,70.600000,60.600000,43.600000,50.600000,44.600000,38.600000,63.600000,...,66.000000,67.800000,5.600000,10.600000,9.600000,7.600000,7.600000,right,medium,medium
1,74.484848,70.787879,49.454545,52.939394,62.272727,29.151515,61.090909,61.878788,62.121212,63.242424,...,68.787879,71.515152,12.181818,8.666667,14.242424,10.363636,12.909091,left,medium,medium
2,74.192308,68.115385,57.923077,58.692308,65.115385,54.269231,69.038462,60.192308,55.615385,60.461538,...,21.115385,21.346154,14.038462,11.807692,17.730769,10.115385,13.500000,right,medium,medium
3,70.782609,57.217391,26.260870,69.260870,64.695652,47.782609,55.565217,37.782609,40.391304,60.826087,...,70.652174,68.043478,14.173913,11.173913,22.869565,11.173913,10.173913,right,medium,medium
4,74.680000,45.080000,38.840000,73.040000,64.760000,32.080000,50.600000,45.480000,26.360000,56.840000,...,76.040000,74.600000,8.280000,8.320000,24.920000,12.840000,11.920000,right,medium,medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11057,75.461538,42.000000,27.000000,75.153846,70.000000,30.000000,56.384615,49.000000,35.692308,54.538462,...,76.307692,72.692308,12.846154,12.769231,25.461538,11.384615,13.615385,right,low,medium
11058,71.625000,67.250000,46.750000,60.312500,63.125000,54.562500,61.187500,63.812500,61.750000,62.312500,...,65.250000,63.875000,9.000000,8.000000,5.000000,5.000000,8.000000,left,medium,medium
11059,72.857143,63.142857,44.571429,59.857143,66.857143,57.000000,66.857143,60.000000,52.571429,58.285714,...,64.285714,59.000000,7.142857,18.714286,42.428571,16.714286,16.714286,left,NaN,_0
11060,78.125000,46.750000,43.000000,79.000000,58.875000,59.000000,36.000000,29.000000,41.375000,50.625000,...,73.250000,65.000000,14.000000,18.500000,41.625000,19.000000,20.000000,right,NaN,_0


In [106]:
merged_player_data.shape

(11062, 37)

In [107]:
# every football player its overall rating (y)
custom_attribute_object = Clean_and_Merge_Target()
merged_target_data = custom_attribute_object.fit_transform(player_data)
print(merged_target_data.head())

   overall_rating
0       63.600000
1       66.969697
2       67.000000
3       69.086957
4       73.240000


In [108]:
merged_target_data.shape

(11060, 1)

In [109]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [110]:
# Ensure indices align between merged_player_data and merged_target_data
merged_player_data, merged_target_data = merged_player_data.align(merged_target_data, join='inner', axis=0)

# Confirm the alignment
print(f"After alignment:")
print(f"merged_player_data: {merged_player_data.shape}")
print(f"merged_target_data: {merged_target_data.shape}")

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(
    merged_player_data, 
    merged_target_data, 
    test_size=0.20, 
    random_state=101
)


After alignment:
merged_player_data: (11060, 37)
merged_target_data: (11060, 1)


In [111]:
#transformer calls the pipelines
num_pipeline = Pipeline([
    ('num_imputer', SimpleImputer(strategy='mean')), 
    ('num_scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('cat_imputer', SimpleImputer(strategy='most_frequent')), 
    ('cat_encoder', OneHotEncoder(handle_unknown='ignore'))
])

full_transformer = ColumnTransformer([
    ('num_transformer', num_pipeline, numercial_attributes),
    ('cat_transformer', cat_pipeline, categorical_attributes)
])

target_pipeline = Pipeline([
    ('num_imputer', SimpleImputer(strategy='mean', add_indicator=True)), 
    ('num_scaler', StandardScaler())
])

In [112]:
full_transformed_data = full_transformer.fit_transform(X_train)
full_transformed_data

array([[ 0.57895488,  0.65690562,  1.21498458, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.75855689,  0.55746641,  1.60947783, ...,  0.        ,
         0.        ,  0.        ],
       [-0.59446693, -0.86961238, -1.07062134, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.02134311, -2.3064447 , -1.75208178, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.77170716, -0.1576683 , -0.54606196, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.91066097,  0.96876225,  0.73932676, ...,  0.        ,
         0.        ,  0.        ]])

In [113]:
full_transformed_target = target_pipeline.fit_transform(pd.DataFrame(y_train))
full_transformed_target

array([[ 0.81114056],
       [ 1.00651404],
       [-0.11224976],
       ...,
       [ 1.23316027],
       [ 0.31232155],
       [ 0.88329677]])

In [114]:
from sklearn.model_selection import cross_val_score

In [115]:
%%time
#linear regression
lin_reg = LinearRegression()

cross_val_score(estimator=lin_reg, X=full_transformed_data,
               y=full_transformed_target, cv=10, n_jobs=-1)


CPU times: user 58 ms, sys: 102 ms, total: 160 ms
Wall time: 1.77 s


array([ 4.79377287e-01,  4.85421472e-01,  3.89293825e-01,  4.23988783e-01,
        4.20506317e-01,  4.68796685e-01,  5.00505173e-01, -3.69784680e+21,
        4.81913484e-01,  4.24483369e-01])

In [116]:
%%time
#decision tree
tree_reg = DecisionTreeRegressor()

cross_val_score(estimator=tree_reg, X=full_transformed_data,
               y=full_transformed_target, cv=10, n_jobs=-1)

CPU times: user 33.3 ms, sys: 15 ms, total: 48.3 ms
Wall time: 2.02 s


array([-0.05121337, -0.0563765 , -0.16685476, -0.09597641, -0.16416423,
       -0.05084194, -0.01438604, -0.15302051, -0.00609641, -0.17374514])

In [117]:
%%time
#SVM
from sklearn.svm import SVR

svm_reg = SVR()

cross_val_score(estimator=svm_reg, X=full_transformed_data,
               y=full_transformed_target, cv=10, n_jobs=-1)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

CPU times: user 165 ms, sys: 31.8 ms, total: 197 ms
Wall time: 20.2 s


array([0.51355384, 0.52421821, 0.39050894, 0.41444497, 0.41546434,
       0.50035738, 0.54661615, 0.44235402, 0.47578291, 0.45399805])

In [118]:
from xgboost import XGBRegressor

xgb_reg = XGBRegressor(gpu_id=0, predictor='gpu_predictor')

cross_val_score(estimator=xgb_reg, X=full_transformed_data,
               y=full_transformed_target, cv=10, n_jobs=-1)

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [21:19:34] WARNING: /workspace/src/common/error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [21:19:34] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [21:19:34] WARNING: /workspace/src/common/error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [21:19:34] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [21:19:34

array([0.46144765, 0.41375243, 0.34855964, 0.41731855, 0.3627119 ,
       0.40840123, 0.45383995, 0.33578056, 0.45469978, 0.41970587])

In [119]:
test_data = full_transformer.transform(X_test)
test_data

array([[ 1.00586143,  0.7197088 ,  1.48896809, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.57170855, -2.10552858, -1.74616366, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.90885343,  1.19543683,  0.04218429, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.96813584, -0.44373891,  0.64475641, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.54091166,  0.71281666, -1.32992263, ...,  0.        ,
         0.        ,  0.        ],
       [-0.04422921,  0.48430218, -0.14235342, ...,  0.        ,
         0.        ,  0.        ]])

In [120]:
# trained models  and get errors(loss) pourcentage of models 
best_model_details = {}
lin_reg.fit(X=full_transformed_data, y=full_transformed_target)

y_pred = lin_reg.predict(test_data)
y_pred = y_pred.reshape(-1, 1)

test_target = target_pipeline.transform(pd.DataFrame(y_test))

error = sqrt(mean_squared_error(y_pred=y_pred, y_true=test_target))
print('Error : ', error)

best_model_details[lin_reg] = error

Error :  0.7276320980610804


In [121]:
tree_reg.fit(X=full_transformed_data, y=full_transformed_target)

y_pred = tree_reg.predict(test_data)
y_pred = y_pred.reshape(-1, 1)

test_target = target_pipeline.transform(pd.DataFrame(y_test))

error = sqrt(mean_squared_error(y_pred=y_pred, y_true=test_target))
print('Error : ', error)

best_model_details[tree_reg] = error

Error :  1.0388629698009972


In [122]:
svm_reg.fit(X=full_transformed_data, y=full_transformed_target.ravel())

y_pred = svm_reg.predict(test_data)
y_pred = y_pred.reshape(-1, 1)

test_target = target_pipeline.transform(pd.DataFrame(y_test))

error = sqrt(mean_squared_error(y_pred=y_pred, y_true=test_target))
print('Error : ', error)

best_model_details[svm_reg] = error

Error :  0.7228875843009591


In [123]:
full_transformed_target.ravel()

array([ 0.81114056,  1.00651404, -0.11224976, ...,  1.23316027,
        0.31232155,  0.88329677])

In [124]:
xgb_reg.fit(X=full_transformed_data, y=full_transformed_target)

y_pred = xgb_reg.predict(test_data)
y_pred = y_pred.reshape(-1, 1)

test_target = target_pipeline.transform(pd.DataFrame(y_test))

error = sqrt(mean_squared_error(y_pred=y_pred, y_true=test_target))
print('Error : ', error)

best_model_details[xgb_reg] = error

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [21:19:46] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [21:19:46] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


Error :  0.7648269509257167


In [125]:
best_model_details

{LinearRegression(): 0.7276320980610804,
 DecisionTreeRegressor(): 1.0388629698009972,
 SVR(): 0.7228875843009591,
 XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...): 0.7648269509257167}

In [126]:
min_value = list(best_model_details.values())[0]
my_model = ''

In [127]:
for key, value in best_model_details.items():
    if value < min_value:
        min_value = value
        my_model = key
        
print('Best performing model : {} \nError : {}'.format(my_model, min_value))

Best performing model : SVR() 
Error : 0.7228875843009591
